# ABS Quarterly National Accounts 5206

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Python-set-up" data-toc-modified-id="Python-set-up-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Python set-up</a></span></li><li><span><a href="#Python-environment" data-toc-modified-id="Python-environment-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Python environment</a></span></li><li><span><a href="#Get-data-from-ABS" data-toc-modified-id="Get-data-from-ABS-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Get data from ABS</a></span></li><li><span><a href="#Plot" data-toc-modified-id="Plot-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Plot</a></span><ul class="toc-item"><li><span><a href="#Growth-charts---based-on-seasonally-adjusted-series" data-toc-modified-id="Growth-charts---based-on-seasonally-adjusted-series-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Growth charts - based on seasonally adjusted series</a></span></li><li><span><a href="#COVID-comparison" data-toc-modified-id="COVID-comparison-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>COVID comparison</a></span></li><li><span><a href="#Unemployment-benefits" data-toc-modified-id="Unemployment-benefits-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Unemployment benefits</a></span></li></ul></li><li><span><a href="#Finished" data-toc-modified-id="Finished-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Finished</a></span></li></ul></div>

## Python set-up

In [1]:
# system imports
import sys
import platform
import psutil
from pathlib import Path
import calendar

# analytic imports
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.units as munits

# abs specific imports
from abs_common import (get_ABS_meta_and_data,
                        get_identifier,
                        finalise_plot,
                        plot_Qgrowth)

# pandas display settings
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

# plotting stuff
plt.style.use('ggplot')

In [2]:
# cache directory
CACHE_DIR = '../cache'
Path(CACHE_DIR).mkdir(parents=True, exist_ok=True)

# charts directory
CHART_DIR = '../charts/5206'
Path(CHART_DIR).mkdir(parents=True, exist_ok=True)

## Python environment

In [3]:
N = 50
print('-'*N)
print(f"System:         {platform.system()}")
print(f"Release:        {platform.release()}")
#print(f"Version:        {platform.version()}")
print(f"Machine:        {platform.machine()}")
print(f"Processor:      {platform.processor()}")
print(f"RAM:            {round(psutil.virtual_memory().total / 1024**3)}GB")
print('-'*N)
print(f"Python:         {platform.python_version()}")
print(f"Psutil:         {psutil.__version__}")
print(f"Pandas:         {pd.__version__}")
print(f"Numpy:          {np.__version__}")
print(f"Matplotlib:     {mpl.__version__}")
print('-'*N)

--------------------------------------------------
System:         Darwin
Release:        20.1.0
Machine:        x86_64
Processor:      i386
RAM:            64GB
--------------------------------------------------
Python:         3.8.3
Psutil:         5.7.0
Pandas:         1.0.5
Numpy:          1.18.5
Matplotlib:     3.2.2
--------------------------------------------------


## Get data from ABS

In [4]:
url_template = ('https://www.abs.gov.au/statistics/economy/national-accounts/'
                'australian-national-accounts-national-income-expenditure-and-product/'
                'MONTH-YEAR/All_time_series_workbooks.zip')

meta, data = get_ABS_meta_and_data(url_template, CACHE_DIR)

File has been cached already
File for sep-2020 of size 5.4 MB


## Plot

### Growth charts - based on seasonally adjusted series

In [5]:
growth_plots = {
    # series: table,
    'Gross domestic product: Chain volume measures ;': '1',
    'GDP per capita: Chain volume measures ;':         '1',
    'Gross value added market sector: Chain volume measures ;': '1',
    'Net domestic product: Chain volume measures ;': '1',
    'Real gross domestic income: Chain volume measures ;': '1',
    'Real gross national income: Chain volume measures ;': '1',
    'Real net national disposable income: Chain volume measures ;': '1',
    'Real net national disposable income: Chain volume measures ;': '1',
}

plot_froms = (None, pd.Timestamp('2018-01-01'))
tags = ('full', 'recent')
series_type = "Seasonally Adjusted"

for plot, table in growth_plots.items():
    for plot_from, tag in zip(plot_froms, tags):

        series, _ = get_identifier(meta, plot,
                                   series_type, 
                                   table)
        
        plot_Qgrowth(data[series], plot, plot_from, 
                     tag, CHART_DIR, 
                     rfooter=f'Source: ABS 5206, table {table}',
                     lfooter=f'Calculated from the {series_type.lower()} series')
        

### COVID comparison

In [6]:
# used for all COVID indexes
start = pd.Timestamp('2018-01-01')
marked = pd.Timestamp('2019-12-01')
m_string = f'{calendar.month_abbr[marked.month]} {marked.year}'
index_base = 100
df = data[data.index >= start]
series_type = "Seasonally Adjusted"

In [7]:
# state final demand - Victoria compared with the rest of Australia
states = {
    # state: table
    'NSW': '26',
    'Vic': '27',
    'Qld': '28',
    'SA':  '29',
    'WA':  '30',
    'Tas': '31',
    'NT':  '32',
    'ACT': '33',
}

HFCE = 'Households ;  Final consumption expenditure - '
CVM = 'Chain volume measures'
plots = meta.loc[
    meta['Data Item Description'].str.contains(HFCE)
    & meta['Data Item Description'].str.contains(CVM)
    & ~meta['Data Item Description'].str.contains('Revisions')
    & (meta['Table'] == states['Vic'])
    & (meta['Series Type'] == series_type)
]['Data Item Description']

for plot in plots:
    rest = []
    for state in states:
        series, units = get_identifier(meta, plot,
                                       series_type, 
                                       states[state])
        if state == 'Vic':
            victoria = df[series]
        else: 
            rest.append(series)
    rest = df[rest].sum(axis=1)
    plotable = pd.DataFrame([victoria, rest]).T
    plotable.columns = ['Victoria', 'Rest of Australia']
    index_row = plotable.loc[marked]
    plotable = plotable.div(index_row) * index_base
    title = plot.replace(HFCE, '')
    title = title.replace(f': Chain volume measures ;', '')
    title = f'Household spending: {title}'

    ax = plotable.plot(lw=2, color=['darkorange', 'blue'])
    finalise_plot(ax, title, 
                  f'Index ({m_string} = {index_base})', 
                  'COVID', CHART_DIR,
                  rfooter=f'Source: ABS 5206; tables {",".join(states.values())}',
                  lfooter=f'{series_type} series {CVM.lower()}')


### Government Benefits Payments

In [16]:
plots = [
    # Data Item Description
    'General government - National ;  Health ;',
    'General government - National ;  Sickness benefits ;',
    'General government - National ;  Benefits to ex-servicemen and their dependants ;',
    'General government - National ;  Permanent disablement benefits ;',
    'General government - National ;  Old age benefits ;',
    'General government - National ;  Unemployment benefits ;', 
    'General government - National ;  Family and child benefits ;',
    'General government - National ;  Family and child benefits ;',
    'General government - National ;  Total social security ;',
    'General government - National ;  Other social assistance benefit payments ;',
    'General government - National ;  Total personal benefits payments ;',
    'General government - State and local ;  Total personal benefits payments ;',
    'General government ;  Total personal benefits payments ;',
]
table = '23'
series_type = 'Original'

for plot in plots:
    id, units = get_identifier(meta, plot, series_type, table)
    if df[id].isna().all():
        continue
    ax = df[id].plot(lw=2, color="#dd0000")
    finalise_plot(ax, plot, f'{units} (Current Prices)', 
                  'COVID', CHART_DIR,
                  rfooter=f'Source: ABS 5206, table {table}',
                  lfooter=f'{series_type} series, ')
        

## Finished

In [9]:
print('Finished')

Finished
